In [ ]:
from datetime import datetime

def make_calc_dir(where_to_make='./'):
    dir_name = f'{datetime.now().strftime("%Y-%m-%d_%H %M-%S_%f")}'
    dir_name = where_to_make + dir_name
    os.makedirs(dir_name, exist_ok=True)
    abs_dir_path = os.path.abspath(dir_name)
    return abs_dir_path

In [ ]:
%%time
%load_ext autoreload
%autoreload 2



import os

os.environ['OMP_NUM_THREADS'] = "2,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+ > PREFIX.out"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.io import read
from ase import units

from ase.geometry.analysis import Analysis
from ase.constraints import FixAtoms, FixBondLengths

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

# import plumed

from ase.calculators.dftb import Dftb

from ase.calculators.espresso import Espresso


calc_directory = make_calc_dir(where_to_make='../results/test/pw-calcs/')
# os.chdir(calc_directory)


ps = 1000 * units.fs

# atoms = read('../data/structures/new_systems/ktu_002.cif')
atoms = read('../results/test/machine_learning/opt.traj', index='-1')


# moving_atoms = [43,47,49,53,57,59,  45,51,55,61]

moving_atoms = [
    #   upper layer rings
    43,47,49,53,57,59, 45,51,55,61, 
    77,73,69,67,83,79, 75,71,85,81,
    33,29,25,23,39,35, 31,27,41,37,
    #   lower layer rings
    76,78,82,66,68,72, 80,84,70,74,
    32,34,38,22,24,28, 36,40,26,30,
    46,42,58,56,52,48, 44,60,54,50
      ]
# fixed_atoms = list( set(range(264)) - set(moving_atoms) )
# fix_atoms = FixAtoms(indices=fixed_atoms)
# # fix_atoms = FixAtoms(indices=[atom.index for atom in atoms if atom.symbol=='N' or atom.symbol=='Si' or atom.symbol=='O'])
# # ch_bonds = Analysis(atoms).get_bonds("C", "H")[0]
# # fix_bond_lengths = FixBondLengths(ch_bonds)
# # atoms.set_constraint([fix_atoms, fix_bond_lengths])
# atoms.set_constraint(fix_atoms)
# calc_base = Dftb(atoms=atoms,
#             label='crystal',
#             kpts=(1,1,1)
#             )


# pseudo_dir = '../../pw-benchmarks/qe/Pseudopotentials/'
# https://www.materialscloud.org/discover/sssp/table/efficiency
pseudo_dir = '../Pseudopotentials/mat_cloud_efficiency/'
pseudopotentials = {
                    # 'Na': 'Na.pbe-tm-gipaw-dc.UPF',
                    # 'Cl': 'Cl.pbe-tm-gipaw.UPF',
                    # 'Cu': 'Cu.pbe-tm-new-gipaw.UPF',
                    # 'Sb': 'sb_pbe_v1.4.uspp.F.UPF', # non GIPAW pseudo!
                    # 'F': 'F.pbe-tm-new-gipaw-dc.UPF',
                    # 'Au': "Au_ONCV_PBE-1.0.oncvpsp.upf", # non GIPAW pseudo!
                    # 'Si': 'Si.pbe-tm-new-gipaw-dc.UPF',
                    # # 'Si': 'Si.pbe-n-rrkjus_psl.1.0.0.UPF',
                    # 'C': 'C.pbe-tm-new-gipaw-dc.UPF',
                    # 'H': 'H.pbe-tm-new-gipaw-dc.UPF',
                    # 'P': "P.pbe-tm-gipaw.UPF",
                    # 'Si': 'Si.pbe-tm-gipaw.UPF',
                    # 'N': 'N.pbe-tm-gipaw.UPF',
                    # 'C': 'C.pbe-tm-gipaw.UPF',
                    # 'H': 'H.pbe-tm-gipaw.UPF',
                    # 'O': 'O.pbe-tm-gipaw.UPF',
                    # 'H': 'H.pbe-rrkjus-gipaw-dc.UPF',
                    # 'C': 'C.pbe-rrkjus-gipaw-dc.UPF',
                    # 'N': 'N.pbe-rrkjus-gipaw-dc.UPF',
                    # 'O': 'O.pbe-rrkjus-gipaw-dc.UPF',
                    # 'Si': 'Si.pbe-n-rrkjus_psl.1.0.0.UPF'
                    "H": "H.pbe-rrkjus_psl.1.0.0.UPF",
                    "C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",
                    "N": "N.pbe-n-radius_5.UPF",
                    "O": "O.pbe-n-kjpaw_psl.0.1.UPF",
                    "Si": "Si.pbe-n-rrkjus_psl.1.0.0.UPF"
                    }


pw_params = {
    'prefix':'crystal', 
    'restart_mode' : 'from_scratch',
    'tstress':True, 
    'tprnfor':True, 
    'nosym':True, 
    'ecutwfc':5, 
    'kpts':(1, 1, 1),
    # 'kpts':None, 
    'ecutrho' : 40,
    # 'occupations' : 'smearing', 
    # 'smearing' : 'gauss', 
    # 'degauss' : 1.0e-2
}

calc = Espresso(
    pseudopotentials=pseudopotentials, 
    pseudo_dir = pseudo_dir,
    outdir = calc_directory + '/outdir', #'../results/outdir',
    directory = calc_directory,
    **pw_params,
                # prefix = 'crystal', restart_mode = 'from_scratch',
                # tstress=True, tprnfor=True, nosym=True, 
                # ecutwfc=10, 
                # # kpts=(1, 1, 1),
                # kpts=None, 
                # ecutrho = 100,
                # occupations = 'smearing', smearing = 'gauss', degauss = 1.0e-2
    )


atoms.calc = calc

# atoms.calc = Plumed(calc=calc_base,
#                     input=setup,
#                     timestep=timestep,
#                     atoms=atoms,
#                     kT=0.1,
#                     log='../results/test/plumed/plumed_log.log')

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen

# atoms.calc = calc_base
# MaxwellBoltzmannDistribution(atoms, temperature_K=300)

dyn = NVTBerendsen(atoms, 0.5 * units.fs, 2000, taut=0.5*1000*units.fs, 
                #    trajectory="../results/test/md_runs/md_test_dft.traj",   
                #    logfile="../results/test/md_runs/md_log_dft.log"
                trajectory=calc_directory + "/md_test_dft.traj",   
                logfile=calc_directory + "/md_log_dft.log"
                   )

dyn.run(300)

# print(atoms.get_forces())

# MaxwellBoltzmannDistribution(atoms, temperature_K=2000)

# dyn = Langevin(atoms, 0.1, temperature_K=0.1/units.kB, friction=0.1,
#                fixcm=True, trajectory='../results/test/machine_learning/dftb_500.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
# dyn = VelocityVerlet(atoms, dt=0.5*units.fs, trajectory='../results/test/machine_learning/dftb_opt_1000_six_rings.traj')  
# https://wiki.fysik.dtu.dk/ase/ase/md.html

# dyn.run(1000)

In [ ]:
atoms.constraints = None

In [ ]:
atoms.get_potential_energy()

In [ ]:
%%time
print(atoms.get_forces())

In [ ]:
from ase.visualize import view

view(atoms)